## Рекомендательные системы

Наша задача — построить рекомендательную систему для сообществ. Будем рекомендовать пользователю только те группы, в которых он ещё не состоял.

– community_id      - id группы  
– description       - описание группы  
– customers_count   - количество участников в группе  
– messages_count    - количество сообщений  
– type              - тип группы  
– region_id         - id региона  
– themeid           - id темы оформления  
– business_category - категория группы  
– business_parent   - родительская категория бизнес-карточки  
– customer_id       - id пользователя  
– status            - статус пользователя в группе  
– join_request_date - дата запроса на вступление  
– deleted           - флаг выхода из группы  

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

from implicit.evaluation import mean_average_precision_at_k
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from lightfm.cross_validation import random_train_test_split

import tqdm
from tqdm.notebook import tqdm_notebook

### Загрузка и обработка данных

In [39]:
X = pd.read_csv('groups_train.csv.gz', sep = '\t')
X

,community_id,customer_id,type,customers_count,messages_count,region_id,themeid,business_category,business_parent,description,join_request_date,status
0,0,60889,7,2966,1,NaN,NaN,NaN,NaN,"Ванга сказала: ""Выживет пчела - выживет челове...",NaN,A
1,2611,60889,7,71102,2,1.040799e+10,293.0,KARAOKE,LEISURE_AND_ENTERTAINMENT,"Ну вот и все, Спасибо за ваше внимания",NaN,P
2,2802,60889,7,6053,1,NaN,NaN,NaN,NaN,"Обсуждение прекрасной,гостеприимной Италии.",NaN,P
3,3610,60889,7,8709,1,NaN,NaN,NaN,NaN,"Только для тех,кто хочет сниматься и поднимать...",NaN,P
4,4496,60889,7,13722,10,NaN,NaN,NaN,NaN,целью создания этой группы является желание со...,NaN,P
...,...,...,...,...,...,...,...,...,...,...,...,...
4272531,100028,638117,7,300,2,NaN,NaN,BULLETIN_BOARD,INTERNET,NaN,NaN,A
4272532,101650,638117,7,197,2,1.039540e+10,NaN,BULLETIN_BOARD,INTERNET,NaN,NaN,A
4272533,107264,638117,7,227,2,NaN,NaN,FAN_CLUB,BLOG,NaN,NaN,A
4272534,109845,638117,7,315,2,NaN,NaN,BULLETIN_BOARD,INTERNET,NaN,NaN,A


In [73]:
X['community_id'].nunique()

149114

In [74]:
X['customer_id'].nunique()

1568446

#### Приведение к числовым индексам

In [31]:
communities = pd.DataFrame(X['community_id'].unique()).rename(columns = {0:'community_id'})
communities['community_id_new'] = communities.index
communities.to_csv('communities.tsv.gz', sep = '\t', compression = 'gzip', index = False)

communities.head(2)

,community_id,community_id_new
0,00dbc35ec26c5b6452a3259194b2d2f74eae7141d2bc75...,0
1,06107f1eae304c45d3e6324cc86f7d39662793a53b159b...,1


In [32]:
customers = pd.DataFrame(X['customer_id'].unique()).rename(columns = {0:'customer_id'})
customers['customer_id_new'] = customers.index
customers.to_csv('customers.tsv.gz', sep = '\t', compression = 'gzip', index = False)

customers.head(2)

,customer_id,customer_id_new
0,947224211267aefcc2e3e9c524fdf46ce329bc638e8bf1...,0
1,b935c3390d82df612de19591d2dd16e1eff8e660746bb3...,1


In [83]:
X = pd.merge(X, communities)
X = pd.merge(X, customers)

X = X.drop(columns=['community_id', 'customer_id'])
X.head(2)

,description,customers_count,messages_count,type,region_id,themeid,business_category,business_parent,status,join_request_date,community_id_new,customer_id_new
0,"Ванга сказала: ""Выживет пчела - выживет челове...",2966,1,7,NaN,NaN,NaN,NaN,A,NaN,0,0
1,Пчеловодство для начинающих и не только. Хотит...,3296,4,7,1.040023e+10,405.0,ANIMALS,PETS,A,NaN,1462,0


In [84]:
X['community_id'] = X['community_id_new']
X['customer_id'] = X['customer_id_new']

In [85]:
X.columns

Index(['description', 'customers_count', 'messages_count', 'type', 'region_id',
       'themeid', 'business_category', 'business_parent', 'status',
       'join_request_date', 'community_id_new', 'customer_id_new',
       'community_id', 'customer_id'],
      dtype='object')

In [86]:
X = X[['community_id', 'customer_id', 'type', 'customers_count', 'messages_count', 'region_id',
    'themeid', 'business_category', 'business_parent', 'description', 'join_request_date', 'status']]

#### Обработка пропусков

In [5]:
X.isnull().sum()[X.isnull().sum()>0]

region_id            2788466
themeid              2155941
business_category     791204
business_parent       791204
description           575377
join_request_date    4232692
dtype: int64

In [40]:
X['description'] = X['description'].fillna("")
X = X.fillna(-1)

In [41]:
X = X.drop(columns = ['join_request_date'])

#### Приведение к числовому формату

In [42]:
X['region_id'] = X['region_id'].astype(int)
X['themeid'] = X['themeid'].astype(int)

In [43]:
X['description_flag'] = X['description'].apply(lambda x: 1 if len(x) > 0 else 0)

#### Кодирование категориальных признаков

In [9]:
X['business_parent'].nunique()

21

In [10]:
X['business_category'].nunique()

231

In [44]:
label_enc_features = ['business_category', 'business_parent']

one_hot = pd.get_dummies(X[label_enc_features])
X = X.drop(columns = label_enc_features)
X = X.join(one_hot)
   
X.head(5)  

,community_id,customer_id,type,customers_count,messages_count,region_id,themeid,description,status,description_flag,...,business_parent_LEISURE_AND_ENTERTAINMENT,business_parent_MEDIA,business_parent_PERSON,business_parent_PETS,business_parent_POPULAR,business_parent_PROFESSIONAL_SERVICES,business_parent_SOCIAL_ORGANISATIONS,business_parent_SPORT,business_parent_TOURISM,business_parent_TRANSPORT
0,0,60889,7,2966,1,-1,-1,"Ванга сказала: ""Выживет пчела - выживет челове...",A,1,...,0,0,0,0,0,0,0,0,0,0
1,2611,60889,7,71102,2,10407994421,293,"Ну вот и все, Спасибо за ваше внимания",P,1,...,1,0,0,0,0,0,0,0,0,0
2,2802,60889,7,6053,1,-1,-1,"Обсуждение прекрасной,гостеприимной Италии.",P,1,...,0,0,0,0,0,0,0,0,0,0
3,3610,60889,7,8709,1,-1,-1,"Только для тех,кто хочет сниматься и поднимать...",P,1,...,0,0,0,0,0,0,0,0,0,0
4,4496,60889,7,13722,10,-1,-1,целью создания этой группы является желание со...,P,1,...,0,0,0,0,0,0,0,0,0,0


In [45]:
from sklearn.preprocessing import LabelEncoder

lab_enc = LabelEncoder()
X['status'] = lab_enc.fit_transform(X['status'])

X['status'].unique()

array([0, 4, 1, 3, 5, 6, 2])

In [46]:
TRAIN_STATUS = X['status']

------

#### Обработка текста

Что сделаем: с помощью встроенной либы sklearn найдём 200 самый частых слов, а затем превратим их в 200 новых колонок, в которых будет содерждаться флаг (число 0 или 1) - встречается указанный тег в описании, или нет.

In [70]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


def str_filter(sym):
    return str.isalpha(sym) or sym == " "

def analyzer(input):
    return ' '.join(''.join(filter(str_filter, input.lower())).split())


freq_vector = CountVectorizer(stop_words = "russian", max_features = 200, analyzer = analyzer)
word_matrix = freq_vector.fit_transform(X['description'].values)

In [72]:
word_matrix.toarray().shape

(4272536, 200)

In [73]:
# Добавляем в датасет 200 колонок с префиксом dscr_

dscr_cols = pd.DataFrame(word_matrix.toarray(), columns = ["dscr_" + str(num) for num in range(200)])
X = pd.concat([X, dscr_cols], axis=1)

X = X.drop(columns = ['description'])
X

,community_id,customer_id,type,customers_count,messages_count,region_id,themeid,status,description_flag,business_category_-1,...,dscr_190,dscr_191,dscr_192,dscr_193,dscr_194,dscr_195,dscr_196,dscr_197,dscr_198,dscr_199
0,0,60889,7,2966,1,-1,-1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2611,60889,7,71102,2,10407994421,293,4,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2802,60889,7,6053,1,-1,-1,4,1,1,...,0,0,0,0,0,0,0,0,0,0
3,3610,60889,7,8709,1,-1,-1,4,1,1,...,0,0,0,0,0,0,0,0,0,0
4,4496,60889,7,13722,10,-1,-1,4,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272531,100028,638117,7,300,2,-1,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4272532,101650,638117,7,197,2,10395404123,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4272533,107264,638117,7,227,2,-1,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4272534,109845,638117,7,315,2,-1,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Применение трансформеров к векторизированным признакам (тегам и отзывам)

Tf-idf-трансформер позволяет оценить важность слова среди остальных - например, если слово встречается практически в каждом
отзыве, оно не очень информативно, и поэтому получит меньшую оценку. С другой стороны, появление редких слов имеет большее
значение, поэтому их оценка выше.

По сути, мы просто заменяем все единички такими оценками.

In [74]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

dscr_tf = TfidfTransformer(norm='l2', sublinear_tf=True, smooth_idf=False)

In [75]:
# Обучаемся на нужных колонках и здесь же преобразуем их

X.iloc[:, X.columns.str.startswith('dscr_')] = dscr_tf.fit_transform(X.iloc[:, X.columns.str.startswith('dscr_')]).todense()
print("fitted ^-^")

fitted ^-^


In [78]:
# Сохраняем транформеры - теперь с их помощью мы можем обрабатывать все данные, в том числе тестовые

import pickle

with open('dscr_tf.pickle', 'wb') as f:
    pickle.dump(dscr_tf, f)
    
print('saved ^-^')   

saved ^-^


------

#### Упрощаем задачу: если пользователь имеет отношение к группе, target = 1

In [80]:
X['target'] = X['status'].apply(lambda x: 0 if x == 0 else 1)
X['target'].value_counts()

X = X.drop(columns = ['status'])

#### Масштабирование признаков

In [81]:
ex_cols = ['community_id', 'customer_id', 'target']

In [82]:
cols_to_scale = [col for col in X.columns if col not in ex_cols]
len(cols_to_scale)

458

In [83]:
scaler = StandardScaler()
X[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])

In [84]:
X

,community_id,customer_id,type,customers_count,messages_count,region_id,themeid,description_flag,business_category_-1,business_category_ACCOUNTING,...,dscr_191,dscr_192,dscr_193,dscr_194,dscr_195,dscr_196,dscr_197,dscr_198,dscr_199,target
0,0,60889,-0.053649,-0.491291,-0.278944,-0.037351,-0.273422,0.394496,2.097628,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,0
1,2611,60889,-0.053649,-0.165813,-0.276906,0.005309,-0.273422,0.394496,-0.476729,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,1
2,2802,60889,-0.053649,-0.476544,-0.278944,-0.037351,-0.273422,0.394496,2.097628,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,1
3,3610,60889,-0.053649,-0.463857,-0.278944,-0.037351,-0.273422,0.394496,2.097628,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,1
4,4496,60889,-0.053649,-0.439910,-0.260604,-0.037351,-0.273422,0.394496,2.097628,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272531,100028,638117,-0.053649,-0.504026,-0.276906,-0.037351,-0.273422,-2.534882,-0.476729,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,0
4272532,101650,638117,-0.053649,-0.504518,-0.276906,0.005257,-0.273422,-2.534882,-0.476729,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,0
4272533,107264,638117,-0.053649,-0.504374,-0.276906,-0.037351,-0.273422,-2.534882,-0.476729,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,0
4272534,109845,638117,-0.053649,-0.503954,-0.276906,-0.037351,-0.273422,-2.534882,-0.476729,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,0


In [85]:
X.to_csv('dataset_TRAIN_prepared.tsv.gz', sep = '\t', compression = 'gzip', index = False)

In [10]:
# X = pd.read_csv('dataset_TRAIN_prepared.tsv.gz', sep = '\t')

In [3]:
X = pd.read_csv('groups_train.csv.gz', sep = '\t')

X['target'] = X['status'].apply(lambda x: -1 if x == -1 else 1)
X['target'].value_counts()

X = X.drop(columns = ['status'])
X.head(3)

,community_id,customer_id,type,customers_count,messages_count,region_id,themeid,business_category,business_parent,description,join_request_date,target
0,0,60889,7,2966,1,NaN,NaN,NaN,NaN,"Ванга сказала: ""Выживет пчела - выживет челове...",NaN,1
1,2611,60889,7,71102,2,1.040799e+10,293.0,KARAOKE,LEISURE_AND_ENTERTAINMENT,"Ну вот и все, Спасибо за ваше внимания",NaN,1
2,2802,60889,7,6053,1,NaN,NaN,NaN,NaN,"Обсуждение прекрасной,гостеприимной Италии.",NaN,1


## Поиск рекомендаций

In [4]:
def show_item(community_id, measure = ""):
    community = X[X['community_id'] == community_id].iloc[0]
    if measure:
        print(f"id = {community['community_id']}, measure = {measure}")
    else:
        print(f"id = {community['community_id']}")
    print()                
    print(f"business_category = {community['business_category']}")
    print(f"business_parent = {community['business_parent']}")
    print("description =", str(community['description']).replace('\n', ' '))
    print('\n')

### 1. Toп-7

In [88]:
top = X['community_id'].value_counts()[:50]

def top_recoms(customer_id):
    customer_groups = list(X[X['customer_id'] == customer_id]['community_id'])
    return [i for i in top.keys() if i not in customer_groups][:7]

In [89]:
top_recoms(60889)

[7781, 20726, 9510, 366, 21120, 8334, 8384]

In [106]:
show_item(7781)

id = 7781

business_category = CREATION
business_parent = BLOG
description = Добрые советы придут на помощь любой хозяюшке. Присоединяйтесь к нашей группе и получайте полезные советы каждый день себе на страницу.




In [107]:
show_item(20726)

id = 20726

business_category = MUSIC
business_parent = BLOG
description = Добро пожаловать в группу "Хорошие Песни". Присоединяйтесь к нам,и приглашайте своих друзей!  В нашей группе есть музыкальные касты на любой праздник.Статусы,клипы,и песни для вашей души. Приятного Вам отдыха и море позитива в нашей с вами группе.




.

### 2. Совстречаемость

In [5]:
X_preprocessed = pd.read_csv('dataset_TRAIN_prepared.tsv.gz', sep = '\t')
X_preprocessed.head(2)

,community_id,customer_id,type,customers_count,messages_count,region_id,themeid,description_flag,business_category_-1,business_category_ACCOUNTING,...,dscr_191,dscr_192,dscr_193,dscr_194,dscr_195,dscr_196,dscr_197,dscr_198,dscr_199,target
0,0,60889,-0.053649,-0.491291,-0.278944,-0.037351,-0.273422,0.394496,2.097628,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,0
1,2611,60889,-0.053649,-0.165813,-0.276906,0.005309,-0.273422,0.394496,-0.476729,-0.007573,...,-0.024554,-0.032031,-0.027216,-0.013398,-0.01628,-0.016289,-0.013695,-0.013842,-0.014618,1


In [109]:
class CoocurrencyRecomendations():
    def __init__(self, Int):
        self.interactions = Int
        
    def coocurrency_count(self):
        Int = self.interactions[["customer_id", "community_id"]].drop_duplicates()
        
        # Для каждого пользователя собираем группы, c которыми он связан
        user_groups = Int.groupby(["customer_id"])["community_id"].apply(list).reset_index()
        
        # Считаем количество групп, с которыми взаимодействовал каждый пользователь
        product_num = [len(i) for i in user_groups["community_id"]]
        user_groups["prod_num"] = product_num
        
        # УБираем тех, кто взаимодействовал с одной группой
        user_groups = user_groups[user_groups["prod_num"] > 1]
        
        # Словарь под группы
        cooc = {}
        for i in tqdm.tqdm_notebook(user_groups.values):
            for j in range(len(i[1])):
                for k in range(len(i[1])):
                    if j != k:
                        try:
                            cooc[str(i[1][j]) + "_" + str(i[1][k])] += 1
                        except:
                            cooc[str(i[1][j]) + "_" + str(i[1][k])] = 1
        cooc_list = []
        for i, j in cooc.items():
            # Если группы встетились вместе больше одного раза, то добавляем в список
            if j != 1:
                cooc_list.append(i.split("_") + [j])
        self.cooc_rec = pd.DataFrame(cooc_list, columns=["item1", "item2", "measure"])
        

    def get_rec(self, i, show=False):
        recs = self.cooc_rec[self.cooc_rec["item1"] == str(i)]\
                            .sort_values("measure", ascending=False)\
                            .head(5)
        print(u"Для сообщества", end = " ")
        show_item(i)

        print(u"Такие рекомендации:\n")   
        
        recoms = recs["item2"].values.astype(int)
        measures = recs["measure"].values.astype(int)
        for r, m in zip(recoms, measures):
            show_item(r, m)

In [113]:
rec = CoocurrencyRecomendations(X_preprocessed)
rec.coocurrency_count()

  0%|          | 0/129500 [00:00<?, ?it/s]

In [114]:
rec.rec.sort_values("measure", ascending=False)

,item1,item2,measure
47026,23105,20542,1839
46716,20542,23105,1839
1190783,11928,394,1740
1187993,394,11928,1740
61449,18073,10893,1640
...,...,...,...
27527954,46043,5413,2
27527945,46043,1302,2
27527941,44654,49011,2
27527940,44654,46043,2


In [116]:
rec.get_rec(30914)

Для сообщества id = 30914

business_category = nan
business_parent = nan
description = разведения и содержание пчел


Такие рекомендации:

id = 39748, measure = 16

business_category = nan
business_parent = nan
 Все о пчелах, их содержании и истории с ними связанные.


id = 2360, measure = 14

business_category = nan
business_parent = nan
description = Создание общества пчеловодов,дает возможность решать проблемы пчеловодства,как для начинающих , так и преуспевающих.


id = 7300, measure = 13

business_category = nan
business_parent = nan
description = Присоединяйтесь и приглашайте своих друзей и знакомых!!! Будем общаться, делиться опытом и узнавать много нового! В группе представлены видео уроки, фото, полезная и интересная информация по пчеловодству, а также новости из мира пчеловодства. Вы можете задать вопрос опытным пчеловодам, принять участие в опросах и оставить комментарии по заинтересовавшей Вас теме. Ссылки на сторонние ресурсы запрещены.


id = 15202, measure = 13

business

In [117]:
rec.get_rec(20726)

Для сообщества id = 20726

business_category = MUSIC
business_parent = BLOG
description = Добро пожаловать в группу "Хорошие Песни". Присоединяйтесь к нам,и приглашайте своих друзей!  В нашей группе есть музыкальные касты на любой праздник.Статусы,клипы,и песни для вашей души. Приятного Вам отдыха и море позитива в нашей с вами группе.


Такие рекомендации:

id = 366, measure = 1592

business_category = MUSICAL_GROUP
business_parent = PERSON
description = Что может быть прекрасней и романтичней  слов, сказанных в стихах... Сайт: https://mfc.chapchap.su/


id = 7781, measure = 1395

business_category = CREATION
business_parent = BLOG
description = Добрые советы придут на помощь любой хозяюшке. Присоединяйтесь к нашей группе и получайте полезные советы каждый день себе на страницу.


id = 17592, measure = 1384

business_category = MUSIC
business_parent = BLOG
description = Творчество, Санкт-Петербург, Россия.  открытки, юмор,  видео, музыка,  гиф. Добро пожаловать! Пусть в нашей группе В

.

### 3. Content Based

In [213]:
from sklearn.metrics.pairwise import cosine_similarity

class ContentBasedRecomendations():
    def __init__(self, Int):
        self.interactions = Int
        self.items_embs = self.interactions.drop(columns = ['customer_id', 'target']).set_index('community_id').drop_duplicates()


    def get_rec_I2I(self, i):
        metrics = cosine_similarity([self.items_embs.loc[i]], self.items_embs)
      
        print(u"Для сообщества", end = " ")
        show_item(i)

        print(u"Такие рекомендации:\n")   
        
        recoms_index = np.argsort(metrics)[0][::-1][0:8]
        measures_index = np.sort(metrics)[0][::-1][0:8]
        for r, m in zip(recoms_index, measures_index):
            show_item(items_embs.index[r], m)
        
        
    def get_rec_U2I(self, i):

        user_int = X_preprocessed[X_preprocessed["customer_id"] == i].iloc[:, 2:-1]
        user_groups = X_preprocessed[X_preprocessed["customer_id"] == i]['community_id']
        user_emb = np.average(user_int, axis=0)
        
        metrics = cosine_similarity([user_emb], self.items_embs)
    
        print("Для пользователя, который состоит в группах:\n")
        for com_id in user_groups:
            show_item(com_id)
        
        print(u"Такие рекомендации:\n")   

        recoms_index = np.argsort(metrics)[0][::-1][0:8]
        measures_index = np.sort(metrics)[0][::-1][0:8]
        for r, m in zip(recoms_index, measures_index):
            show_item(items_embs.index[r], m)

In [214]:
cont_rec = ContentBasedRecomendations(X_preprocessed)

In [215]:
cont_rec.get_rec_I2I(2611)

Для сообщества id = 2611

business_category = KARAOKE
business_parent = LEISURE_AND_ENTERTAINMENT
description = Ну вот и все, Спасибо за ваше внимания


Такие рекомендации:

id = 2611, measure = 0.9999999999999993

business_category = KARAOKE
business_parent = LEISURE_AND_ENTERTAINMENT
description = Ну вот и все, Спасибо за ваше внимания


id = 139669, measure = 0.9954396919605712

business_category = KARAOKE
business_parent = LEISURE_AND_ENTERTAINMENT
description = Мне буд-то бы снился Тот ласковый май, В нём ветер тихонько колышет… То светлое чувство, И жизнь буд-то рай, О нём мы лишь только напишем. Любимое детство ушло стороной, А грёзы сметают метели. Мой ласковый Юра всегда ты со мной, Но свидится мы не успели. Улыбка твоя грела душу мою, Мелодией в сердце звучала. Уходишь мой ангел в другую страну, Чтоб снова начать всё сначала. Любимые строки о белой зиме, О розах, и ночи той темной, О розовом вечере наедине, Мечте что осталась бездомной. Ты будешь любовью в знакомых стихах, И 

In [216]:
cont_rec.get_rec_I2I(20726)

Для сообщества id = 20726

business_category = MUSIC
business_parent = BLOG
description = Добро пожаловать в группу "Хорошие Песни". Присоединяйтесь к нам,и приглашайте своих друзей!  В нашей группе есть музыкальные касты на любой праздник.Статусы,клипы,и песни для вашей души. Приятного Вам отдыха и море позитива в нашей с вами группе.


Такие рекомендации:

id = 20726, measure = 1.0000000000000002

business_category = MUSIC
business_parent = BLOG
description = Добро пожаловать в группу "Хорошие Песни". Присоединяйтесь к нам,и приглашайте своих друзей!  В нашей группе есть музыкальные касты на любой праздник.Статусы,клипы,и песни для вашей души. Приятного Вам отдыха и море позитива в нашей с вами группе.


id = 17592, measure = 0.8815542454460387

business_category = MUSIC
business_parent = BLOG
description = Творчество, Санкт-Петербург, Россия.  открытки, юмор,  видео, музыка,  гиф. Добро пожаловать! Пусть в нашей группе Вам будет уютно и тепло, чтобы хотелось    возвращаться. Приятно

In [219]:
cont_rec.get_rec_U2I(81720)

Для пользователя, который состоит в группах:

id = 1965

business_category = ANIMALS
business_parent = PETS
description = Животные бывают не только домашними или вкусными - они удивительные, захватывающие и даже шокирующие.Подписывайтесь на канал будет много  чего интересного !!!


id = 4791

business_category = ANIMALS
business_parent = PETS
description = Увлечения, общения, красота, любовь и преданность, новости и все самое интересное  о собаках.


id = 6223

business_category = ANIMALS
business_parent = PETS
description = От хвоста и до усов - все узнаешь про котов. Советы любителям кошек http://cats-burg.ru/ Продажа котят) Интересные факты о кошках. Видео про кошек https://www.youtube.com/channel/UCydvn2T6TAocCNxcqUN0ZjQ Сбрасывайте в ленту фото своих любимцев !


id = 8766

business_category = ANIMALS
business_parent = PETS
description = О животных , о природе и красоте мира!


id = 11864

business_category = ANIMALS
business_parent = PETS
description = ПОДАРОК СУДЬБЫ – это благот

-----

### 4. Рекомендации с библиотекой Surprise

In [244]:
import surprise

dir(surprise)

['AlgoBase',
 'BaselineOnly',
 'CoClustering',
 'Dataset',
 'KNNBaseline',
 'KNNBasic',
 'KNNWithMeans',
 'KNNWithZScore',
 'NMF',
 'NormalPredictor',
 'Prediction',
 'PredictionImpossible',
 'Reader',
 'SVD',
 'SVDpp',
 'SlopeOne',
 'Trainset',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'accuracy',
 'builtin_datasets',
 'dataset',
 'dump',
 'get_dataset_dir',
 'get_distribution',
 'model_selection',
 'prediction_algorithms',
 'reader',
 'similarities',
 'trainset',
 'utils']

#### 4.1. Загрузка данных

In [6]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

reader = Reader(sep='\t', rating_scale = (0,1))
data = Dataset.load_from_df(X_preprocessed[['community_id', 'customer_id', 'target']], reader = reader)

In [7]:
x_train, x_test = train_test_split(data, test_size = 0.2)

#### 4.2. Коллаборативная фильтрация на основе кластеризации

In [264]:
from surprise import CoClustering
from surprise.model_selection import cross_validate

co_clustering = CoClustering(n_cltr_u = 100, n_cltr_i = 100, n_epochs = 100, verbose = True, random_state = 42)

co_clustering_results = cross_validate(co_clustering, data, measures=['RMSE', 'MAE'], cv = 3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49
Processing

In [266]:
co_clustering_results

{'test_rmse': array([0.34631212, 0.34510141, 0.34448478]),
 'test_mae': array([0.18914858, 0.18772349, 0.18813728]),
 'fit_time': (2830.5087583065033, 2813.0505402088165, 2851.0500481128693),
 'test_time': (13.613754987716675, 13.86122989654541, 15.06938624382019)}

In [288]:
co_clustering_model = CoClustering(n_cltr_u = 100, n_cltr_i = 100, n_epochs = 50, verbose = True, random_state = 42)

co_clustering_model.fit(x_train)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49


.

In [11]:
def get_recoms_for_user(model, user_id, k = 5, treshold = 0.75):
    user_recoms = []

    for item in x_test:
        predict = model.test([[user_id, item[0], None]])
        if predict[0].est > treshold:
            user_recoms.append((predict[0].iid, predict[0].est))
   
    user_recoms = list(set(user_recoms))
    user_recoms.sort(key = lambda x: x[1], reverse=True)
    return user_recoms[:k]  

In [369]:
user_recoms = get_recoms_for_user(co_clustering_model, 81721)
user_recoms

[(67543, 1), (65444, 1), (54074, 1), (60138, 1), (42197, 1)]

In [370]:
for r in user_recoms:
    show_item(r[0])

id = 67543

business_category = nan
business_parent = nan
description = Друзья приглашаю вступить в группу кто был на вахте на рыбе оставляйте свои отзывы  фото советы истории  пожелания ! Давайте дружить !!!


id = 65444

business_category = ANIMAL_SHELTER
business_parent = PETS
description = В городе Волжском открывается новый приют для брошенных животных🐕🐩🐺. Вопрос проблемы бродячих и брошенных животных в городе стоит очень остро☝. Животных по просту убивают, травят, выбрасывают на улицу как мусор😱. Так уж вышло, что мы живем в городе, где нет специальных служб помощи бездомным животным, нет государственных приютов, нет поддержки местных властей в помощь животным , которые попали в беду. Но есть люди неравнодушные к "бездомным" соседям, люди с добрым сердцем, чувством сострадания и милосердия. В задачи приюта входит спасение, лечение, пожизненное содержание бездомных, безнадзорных животных, поиск им хозяев, стерилизация.


id = 54074

business_category = GOS_ORGANIZATION
business_pa

#### 4.3. K ближайших соседей

In [8]:
data_small = Dataset.load_from_df(X_preprocessed[['community_id', 'customer_id', 'target']][::5], reader = reader)
x_train_small, _ = train_test_split(data_small, test_size = 0.2)

In [9]:
from surprise import KNNWithMeans

knn_means_model = KNNWithMeans(k = 100, verbose = True, random_state = 42)
knn_means_model.fit(x_train)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [12]:
user_recoms = get_recoms_for_user(knn_means_model, 81721, treshold = 0.4)

for r in user_recoms:
    show_item(r[0])

id = 30052

business_category = CREATION
business_parent = BLOG
description = Любители декупажа, собрались  , чтобы обменяться своим опытом, поделиться секретами, и задать интересующие вопросы.


id = 36539

business_category = PRODUCTION
business_parent = PROFESSIONAL_SERVICES
description = «Абаканский трикотаж» - производство трикотажа и реализация трикотажных изделий высокого качества. Основной специализацией нашего предприятия является производство верхнего и бельевого трикотажа для взрослых и детей. Наша миссия — создавать удобную и качественную одежду для повседневной жизни, спорта и активного отдыха.  Трикотаж от производителей «Абаканского трикотажа» хорошо известен покупателям, благодаря разнообразному ассортименту, привлекательному дизайну изделий и низким ценам, по которым мы реализуем оптовый трикотаж.


id = 30522

business_category = nan
business_parent = nan
description = Общение на любые темы.Доска объявлений.Новости, афиша.Юмор.Народный целебник.Кулинария.


id = 43276

#### 4.4. SVD

In [13]:
from surprise import SVD

svd_model = SVD(n_epochs = 50, verbose = True)
svd_model.fit(x_train)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 20
Processing epoch 21
Processing epoch 22
Processing epoch 23
Processing epoch 24
Processing epoch 25
Processing epoch 26
Processing epoch 27
Processing epoch 28
Processing epoch 29
Processing epoch 30
Processing epoch 31
Processing epoch 32
Processing epoch 33
Processing epoch 34
Processing epoch 35
Processing epoch 36
Processing epoch 37
Processing epoch 38
Processing epoch 39
Processing epoch 40
Processing epoch 41
Processing epoch 42
Processing epoch 43
Processing epoch 44
Processing epoch 45
Processing epoch 46
Processing epoch 47
Processing epoch 48
Processing epoch 49


In [30]:
user_recoms = get_recoms_for_user(svd_model, 81723, k = 7)
user_recoms

[(53877, 1),
 (58834, 1),
 (46443, 1),
 (48541, 1),
 (37385, 1),
 (38395, 1),
 (55065, 1)]

In [31]:
for r in user_recoms:
    show_item(r[0])

id = 53877

business_category = GOS_ORGANIZATION
business_parent = GOS_ORGANIZATIONS
description = Муниципальное дошкольное образовательное учреждение Чердаклинский детский сад №1 "Радуга"


id = 58834

business_category = nan
business_parent = nan
description = nan


id = 46443

business_category = nan
business_parent = nan
description = Барча КОСОНСОЙ ахли кушилинглар групага кайнок янгиликладан ва приколладан хабардор булингла


id = 48541

business_category = nan
business_parent = nan
description = RABOTASTROI.RU — на портале размещены вакансии строительных компаний, работодателей, резюме российских соискателей в сфере строительства.


id = 37385

business_category = nan
business_parent = nan
description = Всё самое интересное и важное в Бекабаде. Пишите и делитесь.


id = 38395

business_category = CULTURE_AND_ART
business_parent = BLOG
description = nan


id = 55065

business_category = nan
business_parent = nan
description = Marg'ilon Bank kolleji bitiruvchilarini guruhga taklif

----